In [1]:
import pymysql

cnx = pymysql.connect(user='pgbia20p2g06', password='@mysqlHKG6QY',
                              host='10.126.209.159',
                              database='nos')

import pandas as pd
cli = pd.read_sql("SELECT * FROM client_list_eurekathon", cnx);
profile = pd.read_sql("SELECT * FROM client_profile_data_eurekathon", cnx);

pd.set_option('display.expand_frame_repr', False)

cnx.close()

In [2]:
import pyspark
from pyspark import SparkConf, SparkContext
import pyspark.sql
from pyspark.sql import SparkSession
import os
os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/bin/python3"
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"


spark = SparkSession.builder \
.master("spark://10.126.209.173:7077") \
.appName("grop6") \
.config("spark.executor.instances","8") \
.config("spark.executor.cores","8") \
.config("spark.driver.memory", "5g") \
.getOrCreate()

In [3]:
spark.sparkContext.applicationId

'app-20210314190253-0195'

In [4]:
df_cli=spark.createDataFrame(cli)

In [5]:
df_cli.head()

Row(sa='4329560142D21690C211BCA3EE65C83C', is_donor=nan, month_id='202005')

In [6]:
df_cli.show()

+--------------------+--------+--------+
|                  sa|is_donor|month_id|
+--------------------+--------+--------+
|4329560142D21690C...|     NaN|  202005|
|E516D7AADFF328836...|     NaN|  202005|
|1068EC724654A19CB...|     NaN|  202005|
|92BC24B777B922C3A...|     NaN|  202005|
|01CEC8EC107C6A174...|     NaN|  202005|
|462FED7670A43E013...|     NaN|  202005|
|0C8ED16E9D4C8D028...|     NaN|  202005|
|813FA0126BC8A0CD4...|     NaN|  202005|
|527C39ED63ECA3D59...|     NaN|  202005|
|7064F05F2BAE1235C...|     NaN|  202005|
|B32FCD5129C4E2165...|     NaN|  202005|
|E390CC5F20270FA75...|     NaN|  202005|
|C02AD9C89B6CBBBAA...|     NaN|  202005|
|06854DE868D638E12...|     NaN|  202005|
|C07644DC9DD13BA15...|     NaN|  202005|
|175365940A35670E0...|     NaN|  202005|
|6731987508A54B486...|     NaN|  202005|
|0A5D045DB2C1D2FB4...|     NaN|  202005|
|D7F6F031A7D320F28...|     NaN|  202005|
|35FCB664D19930058...|     NaN|  202005|
+--------------------+--------+--------+
only showing top

In [7]:
df_profile = spark.createDataFrame(profile)

In [8]:
cli2=cli.copy()

In [9]:
%%time
#tempo tradicional
cli2.is_donor=cli2.is_donor.fillna(0)

CPU times: user 6.56 ms, sys: 39 µs, total: 6.6 ms
Wall time: 6.77 ms


In [10]:
%%time
#tempo spark
df_cli=df_cli.na.fill(0,subset='is_donor') #.count()

CPU times: user 1.63 ms, sys: 1.5 ms, total: 3.13 ms
Wall time: 219 ms


In [11]:
30.2/6.78

4.454277286135693

In [12]:
df_profile2 = df_profile[['sa', 'month_id', 'typeh_card_qty',
       'typei_card_qty', 'typea_card_qty', 'typeb_card_qty', 'typec_card_qty',
       'typed_card_qty', 'typee_card_qty', 'typef_card_qty', 'typeg_card_qty',
       'premium_qty']]

In [13]:
from pyspark.sql.types import IntegerType
df_profile2 = df_profile2.withColumn("month_id", df_profile2["month_id"].cast(IntegerType()))

In [14]:
df_cli = df_cli.withColumn("month_id_1", df_cli["month_id"].cast(IntegerType())-1)
#client['month_id_1'] = client['month_id'].astype(int) - 1

In [15]:
df_cli.show()

+--------------------+--------+--------+----------+
|                  sa|is_donor|month_id|month_id_1|
+--------------------+--------+--------+----------+
|4329560142D21690C...|     0.0|  202005|    202004|
|E516D7AADFF328836...|     0.0|  202005|    202004|
|1068EC724654A19CB...|     0.0|  202005|    202004|
|92BC24B777B922C3A...|     0.0|  202005|    202004|
|01CEC8EC107C6A174...|     0.0|  202005|    202004|
|462FED7670A43E013...|     0.0|  202005|    202004|
|0C8ED16E9D4C8D028...|     0.0|  202005|    202004|
|813FA0126BC8A0CD4...|     0.0|  202005|    202004|
|527C39ED63ECA3D59...|     0.0|  202005|    202004|
|7064F05F2BAE1235C...|     0.0|  202005|    202004|
|B32FCD5129C4E2165...|     0.0|  202005|    202004|
|E390CC5F20270FA75...|     0.0|  202005|    202004|
|C02AD9C89B6CBBBAA...|     0.0|  202005|    202004|
|06854DE868D638E12...|     0.0|  202005|    202004|
|C07644DC9DD13BA15...|     0.0|  202005|    202004|
|175365940A35670E0...|     0.0|  202005|    202004|
|6731987508A

In [17]:
from pyspark.sql.window import Window
from pyspark.sql import Row, functions as F

df_profile3 = df_profile2.withColumn("rn", F.row_number().over(Window.partitionBy(["sa","month_id"]).orderBy(F.col("sa").desc())))

In [18]:
df_profile3 = df_profile3.filter(F.col("rn") == 1).drop("rn")

In [19]:
df_profile3.count()

919185

In [20]:
df_profile2.count()

1085546

In [21]:
df_cli2=df_cli.drop('month_id')

In [22]:
dff=df_cli2.join(df_profile3,(df_cli2.sa==df_profile3.sa) & (df_cli2.month_id_1==df_profile3.month_id))

In [23]:
dff.count()

455379

In [24]:
dff.head()

Row(sa='000FD136826DC2E15756F4A94B6D7BD9', is_donor=0.0, month_id_1=202004, sa='000FD136826DC2E15756F4A94B6D7BD9', month_id=202004, typeh_card_qty=0.0, typei_card_qty=2.0, typea_card_qty=0.0, typeb_card_qty=0.0, typec_card_qty=2.0, typed_card_qty=0.0, typee_card_qty=0.0, typef_card_qty=0.0, typeg_card_qty=0.0, premium_qty=1.0)

In [25]:
dff=dff.na.fill(0)

In [26]:
treino = dff[dff.month_id_1 != 202007]
teste = dff[dff.month_id_1 == 202007]

In [27]:
treino.head()

Row(sa='000FD136826DC2E15756F4A94B6D7BD9', is_donor=0.0, month_id_1=202004, sa='000FD136826DC2E15756F4A94B6D7BD9', month_id=202004, typeh_card_qty=0.0, typei_card_qty=2.0, typea_card_qty=0.0, typeb_card_qty=0.0, typec_card_qty=2.0, typed_card_qty=0.0, typee_card_qty=0.0, typef_card_qty=0.0, typeg_card_qty=0.0, premium_qty=1.0)

In [28]:
from functools import reduce
from pyspark.sql import DataFrame

treino=reduce(DataFrame.drop, ['sa', 'month_id', 'month_id_1'], treino)
teste=reduce(DataFrame.drop, ['sa', 'month_id', 'month_id_1'], teste)

In [29]:
treino.head()

Row(is_donor=0.0, typeh_card_qty=0.0, typei_card_qty=2.0, typea_card_qty=0.0, typeb_card_qty=0.0, typec_card_qty=2.0, typed_card_qty=0.0, typee_card_qty=0.0, typef_card_qty=0.0, typeg_card_qty=0.0, premium_qty=1.0)

In [30]:
treino=treino.drop('typei_card_qty')
teste=teste.drop('typei_card_qty')

In [31]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["typeh_card_qty", "typea_card_qty", "typeb_card_qty",
                                      "typec_card_qty", "typed_card_qty", "typee_card_qty",
                                      "typef_card_qty", "typeg_card_qty", "premium_qty"],outputCol="features")

In [32]:
assembler.transform(treino).show(5)

+--------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+--------------------+
|is_donor|typeh_card_qty|typea_card_qty|typeb_card_qty|typec_card_qty|typed_card_qty|typee_card_qty|typef_card_qty|typeg_card_qty|premium_qty|            features|
+--------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+--------------------+
|     0.0|           0.0|           0.0|           0.0|           2.0|           0.0|           0.0|           0.0|           0.0|        1.0| (9,[3,8],[2.0,1.0])|
|     0.0|           1.0|           0.0|           0.0|           3.0|           0.0|           0.0|           0.0|           0.0|        2.0|(9,[0,3,8],[1.0,3...|
|     0.0|           0.0|           0.0|           0.0|           2.0|           0.0|           0.0|           0.0|           0.0|        2.0| (9,[3,8],[2.0,2.0])|
|     0.0|      

In [33]:
from pyspark.ml.feature import StandardScaler

tmpdf = assembler.transform(treino)
#tmpdf.select("features").show(truncate=False)

scaler = StandardScaler(
    withMean=True, withStd=True, inputCol="features", outputCol="scaled"
)

In [34]:
tmpdf2 = assembler.transform(teste)

In [35]:
scale=scaler.fit(tmpdf)

In [36]:
treino2=scale.transform(tmpdf)

In [37]:
teste2=scale.transform(tmpdf2)

In [38]:
teste.show(5)

+--------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+
|is_donor|typeh_card_qty|typea_card_qty|typeb_card_qty|typec_card_qty|typed_card_qty|typee_card_qty|typef_card_qty|typeg_card_qty|premium_qty|
+--------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+
|     0.0|           0.0|           0.0|           0.0|           2.0|           0.0|           0.0|           0.0|           0.0|        0.0|
|     0.0|           0.0|           0.0|           1.0|           2.0|           0.0|           0.0|           0.0|           0.0|        1.0|
|     0.0|           1.0|           0.0|           0.0|           4.0|           0.0|           0.0|           0.0|           0.0|        1.0|
|     0.0|           0.0|           0.0|           0.0|           2.0|           0.0|           0.0|           0.0|           0.0|        0.0|

In [39]:
treino2.select("features","scaled").show(5)

+--------------------+--------------------+
|            features|              scaled|
+--------------------+--------------------+
| (9,[3,8],[2.0,1.0])|[-0.3931936556274...|
|(9,[0,3,8],[1.0,3...|[2.06676327259975...|
| (9,[3,8],[2.0,2.0])|[-0.3931936556274...|
|[2.0,1.0,1.0,6.0,...|[4.52672020082691...|
| (9,[3,8],[2.0,1.0])|[-0.3931936556274...|
+--------------------+--------------------+
only showing top 5 rows



### Logistic Regression

In [40]:
dataset_size=float(treino2.select("is_donor").count())
numNegatives=treino2.select("is_donor").where('is_donor == 0').count()
print('The number of zeros are {}'.format(numNegatives))

The number of zeros are 345261


In [41]:
BalancingRatio= numNegatives/dataset_size
print('BalancingRatio = {}'.format(BalancingRatio))

BalancingRatio = 0.9982940630502731


In [42]:
from pyspark.sql.functions import when
treino2=treino2.withColumn("classWeights", when(treino2.is_donor == 1,BalancingRatio).otherwise(1-BalancingRatio))
#train.select("classWeights").show(5)

In [43]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(labelCol="is_donor", featuresCol="scaled", weightCol="classWeights",maxIter=10)
model=lr.fit(treino2)
predict_train=model.transform(treino2)
predict_test=model.transform(teste2)

In [44]:
predict_test.show(5)

+--------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|is_donor|typeh_card_qty|typea_card_qty|typeb_card_qty|typec_card_qty|typed_card_qty|typee_card_qty|typef_card_qty|typeg_card_qty|premium_qty|            features|              scaled|       rawPrediction|         probability|prediction|
+--------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|     0.0|           0.0|           0.0|           0.0|           2.0|           0.0|           0.0|           0.0|           0.0|        0.0|       (9,[3],[2.0])|[-0.3931936556274...|[0.14872075379442...|[0.53711181082299...|       0.0|
|     0.0|           0.0|           0.0|        

### ROC value

In [45]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="is_donor")

In [46]:
print("The area under ROC for test set is {}".format(evaluator.evaluate(predict_test)))

The area under ROC for test set is 0.6016392003827589


### Times

In [48]:
df_cli=spark.createDataFrame(cli)
df_profile = spark.createDataFrame(profile)

In [49]:
%%time
df_cli=df_cli.na.fill(0,subset='is_donor')

df_profile2 = df_profile[['sa', 'month_id', 'typeh_card_qty',
       'typei_card_qty', 'typea_card_qty', 'typeb_card_qty', 'typec_card_qty',
       'typed_card_qty', 'typee_card_qty', 'typef_card_qty', 'typeg_card_qty',
       'premium_qty']]

df_profile2 = df_profile2.withColumn("month_id", df_profile2["month_id"].cast(IntegerType()))
df_cli = df_cli.withColumn("month_id_1", df_cli["month_id"].cast(IntegerType())-1)

df_profile3 = df_profile2.withColumn("rn", F.row_number().over(Window.partitionBy(["sa","month_id"]).orderBy(F.col("sa").desc())))
df_profile3 = df_profile3.filter(F.col("rn") == 1).drop("rn")

df_cli2=df_cli.drop('month_id')
dff=df_cli2.join(df_profile3,(df_cli2.sa==df_profile3.sa) & (df_cli2.month_id_1==df_profile3.month_id))

dff=dff.na.fill(0)

treino = dff[dff.month_id_1 != 202007]
teste = dff[dff.month_id_1 == 202007]

treino=reduce(DataFrame.drop, ['sa', 'month_id', 'month_id_1'], treino)
teste=reduce(DataFrame.drop, ['sa', 'month_id', 'month_id_1'], teste)

treino=treino.drop('typei_card_qty')
teste=teste.drop('typei_card_qty')

assembler = VectorAssembler(inputCols=["typeh_card_qty", "typea_card_qty", "typeb_card_qty",
                                      "typec_card_qty", "typed_card_qty", "typee_card_qty",
                                      "typef_card_qty", "typeg_card_qty", "premium_qty"],outputCol="features")

tmpdf = assembler.transform(treino)

scaler = StandardScaler(
    withMean=True, withStd=True, inputCol="features", outputCol="scaled"
)

tmpdf2 = assembler.transform(teste)

scale=scaler.fit(tmpdf)
treino2=scale.transform(tmpdf)
teste2=scale.transform(tmpdf2)

dataset_size=float(treino2.select("is_donor").count())
numNegatives=treino2.select("is_donor").where('is_donor == 0').count()
BalancingRatio= numNegatives/dataset_size

treino2=treino2.withColumn("classWeights", when(treino2.is_donor == 1,BalancingRatio).otherwise(1-BalancingRatio))

lr = LogisticRegression(labelCol="is_donor", featuresCol="scaled", weightCol="classWeights",maxIter=10)
model=lr.fit(treino2)
predict_train=model.transform(treino2)
predict_test=model.transform(teste2)

evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="is_donor")

print("The area under ROC for test set is {}".format(evaluator.evaluate(predict_test)))

The area under ROC for test set is 0.6016477241657463
CPU times: user 306 ms, sys: 105 ms, total: 411 ms
Wall time: 1min 53s


In [50]:
spark.stop()

In [54]:
import pymysql

cnx = pymysql.connect(user='pgbia20p2g06', password='@mysqlHKG6QY',
                              host='10.126.209.159',
                              database='nos')

import pandas as pd
client = pd.read_sql("SELECT * FROM client_list_eurekathon", cnx);
prof = pd.read_sql("SELECT * FROM client_profile_data_eurekathon", cnx);

pd.set_option('display.expand_frame_repr', False)

cnx.close()

In [70]:
cli=client.copy()

profile=prof.copy()

In [71]:
%%time

cli['is_donor'].fillna(0,inplace=True)
profile = profile[['sa', 'month_id', 'typeh_card_qty',
       'typei_card_qty', 'typea_card_qty', 'typeb_card_qty', 'typec_card_qty',
       'typed_card_qty', 'typee_card_qty', 'typef_card_qty', 'typeg_card_qty',
       'premium_qty']]

profile.month_id = profile.month_id.astype(int)
cli['month_id_1'] = cli['month_id'].astype(int) - 1
profile=profile.groupby(['sa','month_id']).tail(1)

cli.drop('month_id', axis = 1, inplace = True)
merge_df = cli.merge(profile, how = 'left', left_on = ['sa', 'month_id_1'],  right_on = ['sa', 'month_id'])
merge_df.fillna(0, inplace = True)

treino = merge_df.loc[merge_df.month_id_1 != 202007]
teste = merge_df.loc[merge_df.month_id_1 == 202007]

treino.drop(['sa', 'month_id', 'month_id_1'], axis = 1, inplace = True)
teste.drop(['sa', 'month_id', 'month_id_1'], axis = 1, inplace = True)

treino.drop('typei_card_qty', axis = 1, inplace = True)
teste.drop('typei_card_qty', axis = 1, inplace = True)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(treino.iloc[:, 1:])
X_teste = scaler.transform(teste.iloc[:, 1:])

y_train = treino.iloc[:, 0]
y_teste = teste.iloc[:, 0]

from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression(class_weight = 'balanced',random_state = 0)
model2.fit(X_train, y_train)
prob_results2 = model2.predict_proba(X_teste)

from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(y_teste.values, prob_results2[:,1])
auc_score = auc(fpr, tpr)

print("The area under ROC for test set is", auc_score, "\n")

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


The area under ROC for test set is 0.5834690440179761 

CPU times: user 8.17 s, sys: 3.15 s, total: 11.3 s
Wall time: 5.54 s
